In [2]:
# Importing dependencies
import os
from langchain_openai import ChatOpenAI
from crewai_tools import PDFSearchTool
from crewai_tools import tool


In [3]:
from crewai import Agent, Crew, Process, Task
from langchain.agents import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_huggingface import *

In [4]:
# set GROQ_API_KEY and TAVILY_API_KEY
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
#Define LLM 
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama-3.1-8b-instant",
    temperature=0.1,
    max_tokens=1000,

)


llm1=ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama-3.2-1b-preview",
    temperature=0.1,
    max_tokens=1000,)

In [6]:

# use attentions paper as sample
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response =requests.get(pdf_url)
with open('attentions_is_all_you_need.pdf','wb') as file:
  file.write(response.content)

In [7]:
## Create a RAG tool variable to pass our PDF


rag_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama-3.1-8b-instant",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

/Users/tharhtet/.local/share/virtualenvs/Agentic_RAGs-b0qwRdDV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
result = rag_tool.run("How did self-attention mechanism evolve in large language models?")
print(result)

Using Tool: Search a PDF's content
Relevant Content:
This makes it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 22, 23, 19]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence- aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [28]. To

In [9]:
## Using Tavily API Key for search , try with DuckDuck Go
web_search_tool = TavilySearchResults(k=3)

In [10]:
web_search_tool.run("How did self-attention mechanism evolve in large language models?")

[{'url': 'https://ieeexplore.ieee.org/document/10245906',
  'content': 'Transformers, originally introduced for machine translation, and built upon the Self-Attention mechanism, have undergone a remarkable evolution, establishing themselves as the bedrock of large language models (LLMs). Their unparalleled capacity to model intricate relationships and capture extensive dependencies within sequences has propelled their prominence. This article, presented in a popular science format, serves as an introduction to the transformer architecture, elucidating its [...] Opens in a new window Opens an external website Opens an external website in a new window\n\nThis website utilizes technologies such as cookies to enable essential site functionality, as well as for analytics, personalization, and targeted advertising. To learn more, view the following link:    Privacy Policy\n\nSelf-Attention and Transformers: Driving the Evolution of Large Language Models | IEEE Conference Publication | IEEE X

In [11]:
## Define a Tool
@tool
def router_tool(question):
  """Router"""
  if 'self-attention' in question:
    return 'vectorstore'

  else:
    return 'web-search'

In [12]:
#Agent 1

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web-search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 1
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [14]:
#Agent 2
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)




#Task 2
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
    tools=[rag_tool, web_search_tool],
)

In [16]:
# Crew for the System
simple_rag_crew = Crew(
    agents = [Router_Agent,Retriever_Agent],
    tasks = [router_task,retriever_task],
    verbose= True,
)


test_input = {"question" :  "How U-Net work?"}

#test_input = {"question" :  "Does dorthy have any friends?"}
result1 = simple_rag_crew.kickoff(inputs=test_input)

2025-11-09 00:17:02,498 - 8410570944 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How U-Net work?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: I need to determine whether the question is related to Retrieval-Augmented Generation or not.

Action: router_tool
Action Input: {"question": "How U-Net work?"} 

web-search

Thought: I now know the final answer
Final Answer: websearch

> Finished chain.
 [DEBUG]: == [Router] Task output: websearch


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question How U-Net work? with the help of the respective tool.Use the web_serach_tool to retrieve information

In [33]:
#Agent 3

Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

#Task 3
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [34]:
#Agent 4
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)


#Task4

hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

In [35]:
#Agent 5
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [36]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent,Grader_agent,hallucination_grader],
    tasks=[router_task,retriever_task,grader_task,hallucination_task],
    verbose=True

)

2025-11-02 11:46:43,388 - 8410570944 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [37]:
inputs ={"question":"How does self-attention mechanism help large language models?"}
#inputs ={"question":"How self-attention vs Computer vision"}


In [38]:
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How does self-attention mechanism help large language models?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: I need to determine whether the question is related to Retrieval-Augmented Generation or not.

Action: router_tool
Action Input: {"question": "How does self-attention mechanism help large language models?"} 

vectorstore

Thought: I now know the final answer
Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question How

In [39]:
print(result)

yes


In [40]:
type(result)

str

In [41]:
inputs ={"question":"How self-attention vs Computer vision"}
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question How self-attention vs Computer visionBased on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: I need to determine whether the question is related to Retrieval-Augmented Generation or not.

Action: router_tool
Action Input: {"question": "How self-attention vs Computer vision"} 

vectorstore

Thought: I now know the final answer
Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question How self-attention vs Computer vision with the help